### Data Preprocessing

- description에 구두점 제거
- reviews에 각 리뷰마다 구두점 제거
- descrption이랑 reviews 하나의 리스트로 병합
- 명사만 추출해서 하나의 리스트로 만들기

### Calling review data

In [1]:
import pandas as pd

In [2]:
cafe_reviews = pd.read_csv('data/cafe_reviews.csv')
cafe_reviews.head()

,store,rating,description,reviews
0,오드랑베이커리,4.818182,NaN,"['마농바게트 맛있어요~~', '맛있어요.', '까르보나라 빵 진짜맛있어요', '마..."
1,Bake things better,4.875000,안녕하세요! 베이크 띵스 베러입니다.\n우리 매장은 테이크 아웃을 전문으로 한 베이...,['쿠키만 사려 했는데 스콘이 너무 맛있어보여서… 대체적으로 다 맛있었어요~~ 말차...
2,파파무베이커리,4.656863,아빠가 딸을위해 만드는 건강한빵으로 아이가 아빠를부르는 옹알이의 파파무입니다^^천연...,"['사장님도 친절하고 빵도 맛있어요!', '빵도 케익도 커피도 맛있고 사장님 인성..."
3,텍스처베이커리,4.611111,텍스처 베이커리는 가정식 디저트를 추구합니다.\n꾸며지지 않은 러프한 디저트를 즐겨...,"['키쉬종류가 많아서 좋아요', '매장인테리어 귀엽구 빵도 일반적이지 않은 빵들이 ..."
4,가야베이커리,5.000000,NaN,['규모는 작지만 빵이 맛있고 가격도 저렴해서 자주 방문하고 있습니다. 사장님 친절...


#### description 구두점 제거

In [3]:
import string
import ast

In [4]:
cafe_reviews['description'] = cafe_reviews['description'].fillna('')
cafe_desc = [x.translate(str.maketrans('', '', string.punctuation)) for x in cafe_reviews['description']]
cafe_reviews['description'] = pd.Series(cafe_desc)

#### reviews에 각 리뷰마다 구두점 제거

In [5]:
def clean_reviews(row):
    reviews = [x.translate(str.maketrans('', '', string.punctuation)) for x in row]
    return reviews

In [6]:
reviews = [ast.literal_eval(x) for x in cafe_reviews['reviews']]
cafe_reviews['reviews'] = pd.Series(reviews)
cafe_reviews['reviews'] = cafe_reviews['reviews'].apply(clean_reviews)

#### descrption이랑 reviews 하나의 리스트로 병합

In [7]:
for i in range(len(cafe_reviews)):
    cafe_reviews['reviews'][i].append(cafe_reviews['description'][i])

#### 명사만 남기기

In [23]:
from konlpy.tag import Okt
from tqdm import tqdm

In [27]:
okt = Okt()

In [30]:
def extract_noun(cafe_review):
    reviews_w_noun = []
    for review in tqdm(cafe_review['reviews']):
        store_review = []
        for i in range(len(review)):
            noun = okt.nouns(review[i])
            noun = ' '.join(noun)
            store_review.append(noun)
        reviews_w_noun.append(store_review)
    return reviews_w_noun

In [31]:
t = extract_noun(cafe_reviews)

100%|██████████| 934/934 [01:34<00:00,  9.93it/s]


In [39]:
reviews_in_noun = pd.Series(t)
cafe_reviews['reviews_in_noun'] = reviews_in_noun

### BERTopic

In [ ]:
from bertopic import BERTopic

In [ ]:
text_data = cafe_reviews['reviews']
topic_model = BERTopic(embedding_model='jhgan/ko-sbert-sts')
topics, probs = topic_model.fit_transform(text_data[0])  # test w only one store

In [ ]:
topic_model.visualize_barchart()